#### 사용자 정의 모델 클래스
- 부모 클래스: nn.module
- 필수 오버라이딩
    - __init__(): 모델 구성 즉, 설계
    - forward(): 순방향 학습 진행 코드 구현


In [1]:
# 모듈로딩
import torch                                            # 텐서 관련 모듈
import torch.nn as nn                                   # 인공신경망 관련 모듈
import torch.nn.functional as F                         # 인공신경망 관련 함수들 모듈(손실,활성화 함수)
import torch.optim as optim                             # 최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary                           # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *                   # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *               # 분류 성능 지표 관련 모듈



- [기본] 신경망클래스  <hr>
    - 입력층 -> 입력: 피쳐수 고정
    - 출력층 -> 출력: 타겟수 고정
    - 은닉층 -> 고정
    

In [26]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# 모델 설계
# 데이터셋: 피쳐 4개, 타겟 1개, 회귀
# 입력층: 입력 4개, 출력(퍼셉트론) 20개,   AF ReLU
# 은닉층: 입력 20개, 출력(퍼셉트론) 100개, AF ReLU
# 출력층: 입력 100개, 출력 1개,           AF Sigmoid(2진) & softmax(다중) & X(회귀)

In [33]:
class Mymodul(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드
    def __init__(self, feature_num, in_out, h_out) -> None:
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(feature_num, in_out)   #(4+1)*20= 100 개의 변수
        self.hidden_layer=nn.Linear(in_out, h_out) #(20+1)*100=2100개의 변수
        self.ouput_layer=nn.Linear(h_out,1)   #101

    # forward: 순방향/전방향 학습 진행 시 자동호출되는 메서드(콜백함수-Callback func): 시스템에서 호출됨
    # 전달 인자: 학습할 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) #식이 도출
        F.relu(y)             #AF통해 결과값 반환 (죽은 렐루 생길 가능성O)

        y=self.hidden_layer(y) # 식 도출
        F.relu(y)

        return self.ouput_layer(y) #최종 식 도출



In [78]:
# 은닉층이 동적인 모델
class Mymodul2(nn.Module):
    # 인스턴스/객체 생성 시 자동호출되는 메서드
    def __init__(self, feature_num, in_out, h_list) -> None:
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(feature_num, in_out)   #(4+1)*20= 100 개의 변수
        self.hidden_layer=nn.Linear(in_out, h_list[0])
        for i in range(len(h_list)-1):
            name='hidden_layer'+str(i)
            self.name=nn.Linear(h_list[i],h_list[i+1])
        self.ouput_layer=nn.Linear(h_list[-1],1)   #101

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드(콜백함수-Callback func): 시스템에서 호출됨
    # 전달 인자: 학습할 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) #식이 도출
        F.relu(y)             #AF통해 결과값 반환 (죽은 렐루 생길 가능성O)

        y=self.hidden_layer(y) # 식 도출
        F.relu(y)

        return self.ouput_layer(y) #최종 식 도출

In [83]:
for i in range(len(hidden_list)-1):
    print(hidden_list[i], hidden_list[i+1])

10 20
20 30
30 40
40 50


In [80]:
# 인스턴스 생성
m1=Mymodul(4, 50, 100)
hidden_list=[10,20,30,40,50]
m2=Mymodul2(4, 5, hidden_list)

In [81]:
# 모델 파라미터 확인
for m in m1.parameters():print(m)

Parameter containing:
tensor([[ 0.4821,  0.2508, -0.2644,  0.2276],
        [-0.1710,  0.3457,  0.1973,  0.3884],
        [ 0.4702, -0.2332, -0.2673, -0.2689],
        [ 0.3315,  0.0348,  0.3407,  0.3394],
        [ 0.2080,  0.2115,  0.2604, -0.1670],
        [-0.3498, -0.0207,  0.1196, -0.3210],
        [-0.0703,  0.1198,  0.0680,  0.1732],
        [-0.1902,  0.0087, -0.3969,  0.3766],
        [ 0.1050,  0.0554, -0.2852, -0.4941],
        [ 0.4021, -0.4446,  0.4639, -0.1727],
        [-0.4955,  0.0712, -0.1577, -0.4066],
        [ 0.2237,  0.2059, -0.2839, -0.2782],
        [ 0.2943, -0.0695, -0.4093,  0.4974],
        [-0.2687, -0.4139,  0.1802,  0.3554],
        [ 0.4946,  0.4716,  0.2199,  0.4728],
        [ 0.0345, -0.4329,  0.1914, -0.2744],
        [ 0.3901, -0.2667, -0.4647, -0.3048],
        [-0.0050, -0.3938,  0.3122, -0.3797],
        [ 0.3278, -0.4235,  0.4145,  0.4686],
        [-0.4815, -0.3136, -0.2513,  0.4763],
        [-0.1810,  0.1805,  0.4088, -0.1820],
        [-0.

In [82]:
# 학습진행 -> 모델 인스턴스명(데이터)
dataTS=torch.FloatTensor([[1,3,5,7], [2,4,6,8]]) #모델 클래스의 모델 shape에 맞춰야 함!
targetTS= torch.FloatTensor([[4], [5]])

# 학습
pre_y=m1(dataTS)
print(pre_y)

calling forward()
tensor([[0.1339],
        [0.1097]], grad_fn=<AddmmBackward0>)
